In [1]:
# Import dependencies
import os
import glob
import time
import datetime
import re
from re import search
import copy # might not need this
import pandas as pd
from splinter import Browser
import requests
from bs4 import BeautifulSoup as soup
import random
#import pymongo

In [2]:
def return_dt():
    global current_date
    current_date = str(datetime.datetime.now()).replace(':','.').replace(' ','_')[:-7]
    return current_date
#return_dt()

In [ ]:
# executable_path = {'executable_path': './chromedriver.exe'}

In [ ]:
# url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")

In [3]:
# Step 2)

# Function to ask users if they want to watch the Bot (headless = False) work OR not (headless = True)
# Lastly, will take you directly to the webpage that was inputted
head = ''
browser =''
def head_on_off(executable_path):
    # Have moved two preset variables, head and browser that are both " = '' "
    # assigning these as global variables enable us to reference them outside and inside the function
    global head
    global browser
    # options creates a bound to an answer
    options = [1, 2]
    #executable_path = {'executable_path': './chromedriver.exe'}
    # for all cases where users input in a value that is not valid
    while head not in options:
        head = int(input('Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: '))
        if head not in options:
            print("That was not a valid answer. Please try again. ")
    # For cases where users enter in valid options:
    if head == options[0]:
        print('Head is activated. Please view only the new automated Google Chrome web browser. ')
        print('Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. ')
        browser = Browser('chrome', **executable_path, headless=False)
    if head == options[1]:
        print('Headless mode activated. No web browser will pop up. Please proceeed. ')
        browser = Browser('chrome', **executable_path, headless=True)
    # visit the target site
    browser.visit(url)
    global current_url
    current_url = browser.url
    #print(current_url)
    return current_url

#head_on_off(executable_path)
#time.sleep(5)

In [ ]:
# Step 3)

# Use Splinter to grab the current url, to setup request to pull URL
#current_url = browser.url #+ '&Page=' #+ str(turn_page)

# Use Request.get() to pull the current url
response = requests.get(current_url)
response

In [ ]:
# Step 4)

# Use BeautifulSoup to grab all the HTML using the htmlparser
current_page_soup = soup(response.text, 'html.parser')

In [ ]:
#current_page_soup.find_all("a", class_="item-title")[0].text
current_page_soup.find_all("div", class_="item-container")

In [4]:
# Step 5) Are there scrappable items-contrainers on the page? List first, last and count, also how many pages

def scrappable_y_n(current_page_soup):
    global containers
    containers = current_page_soup.find_all("div", class_="item-container")
    
    # print first and last objects so users can understand what the output will be
    print("Here are the items that you can expect to be scrapped off this page, and for every other total results pages, if there's more than one: ")
    print("="*35)
    # max items should be 36
    counter = 0
    for con in containers:
        try:
            counter += 1
            product_details = con.find_all("a", class_="item-title")[0].text
            product_price = con.find_all("li", class_="price-current")[0].text.split()[0]
            print(f'{counter}) {product_details} | Price: {product_price}')
            print("-"*35)
            
        except (IndexError) as e:
            print(f"{counter}) This item was not scrappable. Skipped. ")
            print("-"*35)
            
    print("="*60)
    if counter == 0:
        print("Unable to scrap this link. ")
    else:
        print(f"{len(containers)} Scrappable Objects on the page. ")
    #return containers
#scrappable_y_n(current_page_soup)

In [ ]:
# Create basic classes here and then have the function create product objects AND export out to CSV

In [5]:
def newegg_page_scraper(containers, turn_page): #before: (containers, turn_page)
    
    images = []
    product_brands = []
    product_models = []
    product_links = []
    #item_numbers = []
    product_categories = []
    promotions = []
    prices = []
    shipping_terms = []
    page_nums = []


    for con in containers:
        try:
            page_counter = turn_page
            page_nums.append(int(turn_page))

            image = con.a.img["src"]
            #print(image)
            images.append(image)

            prd_title = con.find_all('a', class_="item-title")[0].text
            product_models.append(prd_title)

            product_link = con.find_all('a', class_="item-title")[0]['href']
            product_links.append(product_link)

            shipping = con.find_all('li', class_='price-ship')[0].text.strip().split()[0]
            if shipping != "Free":
                shipping = shipping.replace('$', '')
                shipping_terms.append(shipping)
            else:
                shipping = 0.00
                shipping_terms.append(shipping)

            brand_name = con.find_all('a', class_="item-brand")[0].img["title"]
            product_brands.append(brand_name)

        except (IndexError, ValueError) as e:
            # if there's no item_brand container, take the Brand from product details
            product_brands.append(con.find_all('a', class_="item-title")[0].text.split()[0])
            #print(f"{e} block 1")

        try:
            current_promo = con.find_all("p", class_="item-promo")[0].text
            promotions.append(current_promo)
        except:
            promotions.append('null')
            #print(f"{e} block 2")
        try:
            price = con.find_all('li', class_="price-current")[0].text.split()[0].replace('$','').replace(',', '')
            prices.append(price)
        except (IndexError, ValueError) as e:
            prices.append('null')
            #print(f"{e} block 3")


    df = pd.DataFrame({
    #'item_number': item_numbers,
    'brand': product_brands,
    'model_specifications': product_models,
    'price': prices,
    'current_promotions': promotions,
    'shipping': shipping_terms,
    'page_number': page_nums,
    'product_links': product_links,
    'image_link': images
    })

    df['product_category'] = current_page_soup.find_all('h1', class_="page-title-text")[0].text
    # rearrange columns
    df = df[['product_category' ,'brand','model_specifications' ,'current_promotions' ,'price' ,'shipping' ,'product_links','image_link']]
    global product_category
    product_category = df['product_category'].unique()[0]
    # eliminate special characters in a string if it exists
    product_category = ''.join(e for e in product_category if e.isalnum())
    
    #return_list.append(product_category)
    global items_scraped
    items_scraped = len(df['model_specifications'])
    
    df.to_csv(f'./processing/{current_date}_{product_category}_{items_scraped}_scraped_page{turn_page}.csv')
    return items_scraped, product_category
    
#df.head()
#newegg_page_scraper(containers, turn_page)

In [ ]:
#print(containers[1].find_all('a', class_="item-brand")[0].img["title"])

In [6]:
#######################################################################################


# create a function to return results pages, if exists, otherwise just scrape one page
def results_pages():
    # Use BeautifulSoup to extract the total results page number
    results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #print(results_pages)
    # Find and extract total pages + and add 1 to ensure proper length of total pages
    global total_results_pages
    total_results_pages = int(re.split("/", results_pages)[1]) #+ 2 # need to add 2 b/c 'range(inclusive, exclusive)'
    #========================================= need to remember to +2, and remove -30
    #print(total_results_pages)
    total_results_pages = total_results_pages ## ### ## ###
    
    return total_results_pages
#results_pages()

In [7]:
# Working
def concatenate(total_results_pages):
    
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    concatenate_pages = []
    counter = 0
    for page in scraped_pages:
        df = pd.read_csv(page, index_col=0, header=0)
        concatenate_pages.append(df)

    compiled_data = pd.concat(concatenate_pages, axis=0, ignore_index=True)
    total_items_scraped = len(compiled_data['brand']) # can replace this counter by creating class objects everytime it scrapes
    concatenated_output = compiled_data.to_csv(f"./finished_outputs/{current_date}_{total_items_scraped}_scraped_{total_results_pages}_pages_.csv")
    return concatenated_output


In [ ]:
# total_results_pages = 4
# concatenate(total_results_pages)

In [8]:
# THis is working
def clean_processing_fldr():
    # delete all files in the 'processing folder'
    path = f'./processing\\'
    scraped_pages = glob.glob(path + "/*.csv")
    if len(scraped_pages) < 1:
        print("There are no files in the folder to clear. ")
    else:
        print(f"Clearing out a total of {len(scraped_pages)} scraped pages in the processing folder... ")
        clear_processing_files = []
        for page in scraped_pages:
            os.remove(page)
        
    print('Clearing of "Processing" folder complete. ')

In [ ]:
# webscrape first page, then run page turner, then scraper for every page thereafter
# 
    

In [9]:
# learning lesson is you can't call a function within a function

def page_turner(total_results_pages):
    # This is "NEXT PAGE BUTTON CLICK" - This loops thru the total amount of pages by clicking the next page button
    for turn_page in range(1, total_results_pages):
        # set the current url as the target page (aiming the boomerang)
        target_url = browser.url

        # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
        response_target = requests.get(target_url)
        #response

        # Use BeautifulSoup to read grab all the HTML using the lxml parser
        target_page_soup = soup(response_target.text, 'html.parser')

        # Use BeautifulSoup to extract the total results page number
        #results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
        
        results_pages = target_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
        #=========================================================
        containers = target_page_soup.find_all("div", class_="item-container")
        
        newegg_page_scraper(containers, turn_page)
        
        #for i in range(total_results_pages):
        x = random.randint(3, 25)
        print(f"{turn_page}) | SLEEPING FOR SECONDS: {x} ")
        time.sleep(x)
            
        browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

    browser.quit()
# concatenate(total_results_pages)
# clean_processing_fldr()
# # clear out processing folder function here - as delete everything to prevent clutter


# print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{product_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')
# print('Thank you and hope you found this useful!')

In [10]:
# scrape_again = 'y'
# while scrape_again =='y':
#     print("=== NewEgg.Com WebScraper Beta ===")
#     print("=="*30)
#     print("Instructions:")
#     print("(1) Go to www.newegg.com, search for the product and filter for your requirements (e.g. brand and specifications) ")
#     print("(2) Copy and paste the url from your exact search ")
#     print('(3) Activate or Disable the "Head View", webscraper bots point of view ')
#     print('(4) Check the "final_output folder when the webscraper bot is done scraping "')
#     print("")
# # NEED TO CREATE LAST INPUT TO LOOP THIS WHOLE THING AGAIN.
# Maybe use pandas to provide summary of data
# Export images of box and whisker plots using statistics by brand

return_dt()

executable_path = {'executable_path': './chromedriver.exe'}

url = input("Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: ")
head = ''
browser =''
head_on_off(executable_path)
response = requests.get(current_url)
#response

current_page_soup = soup(response.text, 'html.parser')
current_page_soup.find_all("div", class_="item-container")
scrappable_y_n(current_page_soup)

# Are there any pop ups / safe to proceed?
safe_proceed_y_n = input(f'The Break Pedal: Answer any robot queries by NewEgg. Enter "y" when you are ready to proceed. ')
if safe_proceed_y_n == 'y':
    print(f'Proceeding with webscrape... ')
else:
    print("Quitting browser. You will need to press ctrl + c to quit, and then restart the program to try again. ")
    browser.quit()
#newegg_page_scraper(containers)

# will need to UNCOMMENT AFTER
#results_pages()

#page_turner(total_results_pages)

total_results_pages = 5
for turn_page in range(1, total_results_pages):
    # set the current url as the target page (aiming the boomerang)
    target_url = browser.url

    # Use Request.get() - throw the boomerang at the target, retrieve the info, & return back to requestor
    response_target = requests.get(target_url)
    #response

    # Use BeautifulSoup to read grab all the HTML using the lxml parser
    target_page_soup = soup(response_target.text, 'html.parser')

    # Use BeautifulSoup to extract the total results page number
    #results_pages = current_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()

    results_pages = target_page_soup.find_all('span', class_="list-tool-pagination-text")[0].text.strip()
    #=========================================================
    containers = target_page_soup.find_all("div", class_="item-container")

    newegg_page_scraper(containers, turn_page)

    #for i in range(total_results_pages):
    x = random.randint(3, 25)
    print(f"{turn_page}) | SLEEPING FOR {x} SECONDS ")
    time.sleep(x)

    browser.find_by_xpath('//*[@id="bodyArea"]/section/div/div/div[2]/div/div/div/div[2]/div[1]/div[2]/div[1]/div[2]/div/div[2]/button').click()

browser.quit()


##################################################################
concat_y_n = input(f'All {total_results_pages} pages have been saved in the "processing" folder (1 page = csv files). Would you like for us concatenate all the files into one? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if concat_y_n == 'y':
    concatenate(total_results_pages)
    print(f'WebScraping Complete! All {total_results_pages} have been scraped and saved as {current_date}_{product_category}_scraped_{total_results_pages}_pages_.csv in the "finished_outputs" folder')

# clear out processing folder function here - as delete everything to prevent clutter
clear_processing_y_n = input(f'The "processing" folder has {total_results_pages} csv files of each page that was scraped. Would you like to clear the files? Enter "y", if so. Otherwise, enter anykey to exit the program. ')
if clear_processing_y_n == 'y':
    clean_processing_fldr()

    
print('Thank you and hope you found this useful!')

Step 1) Please copy and paste your laptop query that you want to webscrape, and press enter: https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32?Tid=6740
Do you want the desktop watch the bot work? Enter a number: 1 - YES | 2 - NO . Your Answer: 1
Head is activated. Please view only the new automated Google Chrome web browser. 
Do not make any adjustments to this automated window while the program runs, as it may produce errors or undesired outputs. 
Here are the items that you can expect to be scrapped off this page, and for every other total results pages, if there's more than one: 
1) GIGABYTE AERO 15 Classic-SA-U74ADP Gaming Laptop - 15.6" 4K/UHD IPS - Intel Core i7-9750H 2.60 GHz - NVIDIA GeForce GTX 1660 Ti - 16 GB Memory 512 GB SSD - Windows 10 Pro 64-bit | Price: $1,379.00
-----------------------------------
2) ASUS VivoBook S15 S532 Thin & Light 15.6" FHD, Intel Core i7-8565U CPU, 8 GB DDR4 RAM, PCIe NVMe 512 GB SSD, Windows 10 Home, S532FA-SB77, Transparent Silver | Pr

The Break Pedal: Answer any robot queries by NewEgg. Enter "y" when you are ready to proceed. y
Proceeding with webscrape... 
1) | SLEEPING FOR SECONDS: 10 
2) | SLEEPING FOR SECONDS: 17 
3) | SLEEPING FOR SECONDS: 17 
4) | SLEEPING FOR SECONDS: 5 
All 5 pages have been saved in the "processing" folder (1 page = csv files). Would you like for us concatenate all the files into one? Enter "y", if so. Otherwise, enter anykey to exit the program. y
WebScraping Complete! All 5 have been scraped and saved as 2020-04-12_19.25.11_LaptopsNotebooks_scraped_5_pages_.csv in the "finished_outputs" folder
The "processing" folder has 5 csv files of each page that was scraped. Would you like to clear the files? Enter "y", if so. Otherwise, enter anykey to exit the program. n
Thank you and hope you found this useful!
